# vision model:
models that extract data from images.

input1 : image
input2 : prompt -"Where is the photographer resting?"

In [ ]:
image -> contrastive vision encoder -> linear projection ->embeddingImage

prompt -> tokenizer( sentencePiece) ->embedddingPrompt

=> concatenated embedding (embeddingImage + embedddingPrompt) -> transformer decoder -> response

Each input image will be split into  blocks  of pixels creating a grid; then each block will be converted into an embedding. This embedding is a vector of fixed size.. that will be concenated with tokens' embeddings of  input prompt text i.e. each token will have an embedding just as each block of pixels will have an embedding. then this concatenated vetor will be sent to transformer decoder.

![alt text](vision_lang_model_01.png)


## Image encoder

### Contrastive learning

![alt text](vision_lang_model_02.png)

---


**Image encoder is vision transformer**

each I1 is in the embedding of image 1


**so for n image In with n descriptions Tn, we will get n*n matrix**



**we want dot prduct of image It with corresponding description text Tt to give higher value**
**and dot product of non corrspnding image and description should be low**
***so, we want diagonal to have high values and all non-diagonal low.***

### what is contrastive learninig?
so with contrastive learning we take list of images and a list of corresponding texts; we encoder them and then we want to train text encoer and image encoder to produce embeddings in such a way that dot product of image with its corresponding text is done, it should produce high value and the dot product of image with non-corresponding text should retunr lower value


**What is contrastive learning?**  
Contrastive learning is a machine learning technique where we work with pairs of related data, such as images and their corresponding text descriptions. The goal is to train two encoders—a text encoder and an image encoder—so that they generate embeddings (numerical representations) with specific properties:  
- The dot product of an image embedding and its corresponding text embedding should be high, indicating strong similarity.  
- Conversely, the dot product of an image embedding with a non-corresponding text embedding should be low, indicating dissimilarity.  

This approach helps the model learn to associate related data while distinguishing unrelated pairs effectively.  

**all non corresponding images are -ve samples**

Here’s a more technical and detailed explanation:  

**What is Contrastive Learning?**  
Contrastive learning is a representation learning paradigm that trains models by distinguishing between similar and dissimilar data pairs. It is widely used in tasks where the goal is to map data from different modalities (e.g., images and text) into a shared embedding space.  

In the context of image-text contrastive learning, the process involves:  

1. **Input Data**:  
   - A dataset consisting of pairs of images \( I \) and their corresponding textual descriptions \( T \).  
   - The dataset also implicitly includes negative pairs, where an image \( I \) is matched with a non-corresponding text \( T' \).  

2. **Encoders**:  
   - An **image encoder** \( f_I(I) \): Maps images into a high-dimensional embedding space. This is often a convolutional neural network (e.g., ResNet, Vision Transformer).  
   - A **text encoder** \( f_T(T) \): Maps text descriptions into the same embedding space. This is often a Transformer-based model (e.g., BERT, RoBERTa).  

3. **Objective**:  
   - The goal is to learn embeddings \( \mathbf{z}_I = f_I(I) \) for images and \( \mathbf{z}_T = f_T(T) \) for text such that:  
     - The **similarity score** (e.g., dot product or cosine similarity) between embeddings of corresponding pairs \( (\mathbf{z}_I, \mathbf{z}_T) \) is maximized.  
     - The similarity score between embeddings of non-corresponding pairs \( (\mathbf{z}_I, \mathbf{z}_{T'}) \) is minimized.  

4. **Loss Function**:  
   - A popular loss for contrastive learning is the **InfoNCE loss** (based on Noise Contrastive Estimation):  
     \[
     \mathcal{L} = - \frac{1}{N} \sum_{i=1}^{N} \left[ \log \frac{\exp(\text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_i}) / \tau)}{\sum_{j=1}^{N} \exp(\text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_j}) / \tau)} \right]
     \]  
     Here:  
     - \( \text{sim}(\mathbf{z}_{I}, \mathbf{z}_{T}) \) is the similarity measure (e.g., dot product or cosine similarity).  
     - \( \tau \) is a temperature hyperparameter that controls the sharpness of the distribution.  
     - \( N \) is the batch size.  
     - The numerator represents the similarity of the positive pair, while the denominator sums over similarities for all pairs in the batch (positive and negative).  

5. **Training Dynamics**:  
   - The encoders are trained jointly to minimize the contrastive loss. This ensures that embeddings of positive pairs are pulled closer together in the shared embedding space, while embeddings of negative pairs are pushed farther apart.  

6. **Applications**:  
   - Contrastive learning is foundational in models like **CLIP** (Contrastive Language-Image Pretraining), where it is used to align visual and textual modalities.  
   - It is also used in self-supervised learning frameworks (e.g., SimCLR, MoCo) to learn representations without explicit labels by treating augmentations of the same image as positive pairs and different images as negative pairs.  

This approach is highly effective for multimodal tasks, enabling downstream applications like image-text retrieval, zero-shot classification, and multimodal embedding alignment.  

Let me know if you’d like even more depth on any specific part!

### WE do this using cross entropy loss in contrastive laerning 


Using CEloss we can force a number(true label ) to have larger value. we will be cosidering vertical for text and horizontql for images

isn't CEloss just like a look up table where only true label value is considered? Remember


### Cross-Entropy Loss in Contrastive Learning  

In contrastive learning, **Cross-Entropy Loss (CE Loss)** is often employed to enforce alignment between corresponding pairs (e.g., images and texts) and separation between non-corresponding pairs. Here's a detailed explanation:

---

### **How Cross-Entropy Loss Works in Contrastive Learning**  

1. **Embedding Alignment**:  
   - We have embeddings for **images** and **texts**. For simplicity:  
     - Let \( \mathbf{z}_I \) represent the embedding of an image.  
     - Let \( \mathbf{z}_T \) represent the embedding of a text.  
   - The similarity between an image-text pair is computed, often using the **dot product** or **cosine similarity**.

2. **Similarity Matrix**:  
   - For a batch of \( N \) image-text pairs, we calculate the similarity scores for all pairs, forming a similarity matrix \( S \in \mathbb{R}^{N \times N} \):  
     \[
     S[i, j] = \text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_j})
     \]  
     - The \( i^{th} \) row corresponds to similarities between the \( i^{th} \) image and all texts in the batch.  
     - The \( j^{th} \) column corresponds to similarities between the \( j^{th} \) text and all images in the batch.

3. **Cross-Entropy Loss Objective**:  
   - Cross-Entropy Loss forces the model to focus on the **true labels** by maximizing the similarity score of the corresponding pair (positive pair) while minimizing the similarity scores for non-corresponding pairs (negative pairs).  
   - The CE Loss for the image-to-text direction can be written as:  
     \[
     \mathcal{L}_{I \to T} = - \frac{1}{N} \sum_{i=1}^N \log \frac{\exp(S[i, i] / \tau)}{\sum_{j=1}^N \exp(S[i, j] / \tau)}
     \]  
     Similarly, for the text-to-image direction:  
     \[
     \mathcal{L}_{T \to I} = - \frac{1}{N} \sum_{j=1}^N \log \frac{\exp(S[j, j] / \tau)}{\sum_{i=1}^N \exp(S[i, j] / \tau)}
     \]  
     - \( S[i, i] \): Similarity of the positive pair (image \( i \) with text \( i \)).  
     - \( \tau \): Temperature parameter controlling sharpness.  
     - The numerator focuses on the true label (positive pair), while the denominator aggregates over all possible labels (negative pairs included).

4. **Loss Aggregation**:  
   - The total loss is a combination of both directions:  
     \[
     \mathcal{L} = \mathcal{L}_{I \to T} + \mathcal{L}_{T \to I}
     \]  

---

### **Is Cross-Entropy Loss Like a Lookup Table?**  

In a way, **Cross-Entropy Loss acts like a lookup table** because it explicitly focuses on the **true label's score** (the positive pair) while treating other scores as negatives. However, it's more sophisticated than a simple lookup table:  

1. **Focus on the True Label**:  
   - CE Loss forces the score of the true label (positive pair) to dominate the similarity matrix, making it larger than all other scores.  

2. **Gradient-Based Optimization**:  
   - Unlike a static lookup table, CE Loss generates gradients for both the positive pair (to increase its similarity) and all negative pairs (to decrease their similarity).  
   - This dynamic adjustment ensures that the embeddings are optimized in a shared space.  

3. **Batch-Wide Comparison**:  
   - CE Loss considers the entire batch for computing the loss, encouraging embeddings to learn relationships relative to all samples in the batch.  

---

### **"Vertical for Text and Horizontal for Images"**  

This refers to how we interpret the similarity matrix \( S \):  
- **Rows (horizontal)**: Each row corresponds to one image and its similarity with all texts.  
- **Columns (vertical)**: Each column corresponds to one text and its similarity with all images.  

For contrastive learning:  
- Each **row** is used to compute the image-to-text loss (\( \mathcal{L}_{I \to T} \)).  
- Each **column** is used to compute the text-to-image loss (\( \mathcal{L}_{T \to I} \)).  

---

### Summary  

Cross-Entropy Loss in contrastive learning isn't just a lookup table—it’s a dynamic mechanism that enforces alignment for positive pairs and separation for negative pairs across the entire batch. It uses gradients to optimize the embeddings iteratively, ensuring the model learns meaningful relationships between modalities (e.g., images and text).

![alt text](vision_lang_model_03_code.png)

**labels = np.arrange(n) beccause of diagonal sequence i.e. 0,1,2,3...**